# スキルデータを作成

In [ ]:
# 練成で出るスキルをコスト順に管理

SKILL_DATA = {
    "3コスト": [
        "ひるみ軽減", "スタミナ奪取", "火属性攻撃強化", "水属性攻撃強化",
        "氷属性攻撃強化", "雷属性攻撃強化", "龍属性攻撃強化", "滑走強化",
        "笛吹き名人", "防御", "精霊の加護", "体力回復量ＵＰ",
        "回復速度", "風圧耐性", "属性やられ耐性", "毒耐性",
        "麻痺耐性", "睡眠耐性", "気絶耐性", "泥雪耐性",
        "爆破やられ耐性", "砥石使用高速化", "アイテム使用強化", "広域化",
        "満足感", "不屈", "腹減り耐性", "飛び込み",
        "陽動", "乗り名人", "供応", "壁面移動【翔】"
    ],
    "6コスト": [
        "砲弾装填", "ガード性能", "ガード強化", "毒属性強化",
        "麻痺属性強化", "睡眠属性強化", "爆破属性強化", "ランナー",
        "体術", "スタミナ急速回復", "抜刀術【力】", "納刀術",
        "ＫＯ術", "特殊射撃強化", "ブレ抑制", "早食い",
        "耐震", "泡沫の舞", "回避性能", "回避距離ＵＰ",
        "破壊王", "壁面移動", "逆襲", "顕如盤石",
        "装填速度", "反動軽減", "霞皮の恩恵", "鋼殻の恩恵",
        "炎鱗の恩恵", "煽衛", "剛心"
    ],
    "9コスト": [
        "剛刃研磨", "キノコ大好き", "心眼", "会心撃【属性】",
        "弾導強化", "抜刀術【技】", "集中", "強化持続",
        "攻めの守勢", "耳栓", "火事場力", "鬼火纏",
        "翔蟲使い", "チャージマスター", "攻勢", "研磨術【鋭】",
        "刃鱗磨き", "合気", "弱点特効【属性】", "巧撃",
        "激昂", "状態異常確定蓄積", "龍気活性", "業鎧【修羅】",
        "奮闘", "粉塵纏", "風纏", "狂竜症【翔】"
    ],
    "12コスト": [
        "匠", "挑戦者", "フルチャージ", "逆恨み",
        "死中に活", "災禍転福", "力の解放", "渾身",
        "幸運", "連撃", "チューンアップ", "高速変形",
        "砲術", "闇討ち", "血氣", "狂竜症【蝕】",
        "蓄積時攻撃強化", "冰気錬成", "龍気変換"
    ],
    "15コスト": [
        "見切り", "超会心", "弱点特効", "達人芸",
        "業物", "弾丸節約", "通常弾・連射矢強化", "貫通弾・貫通矢強化",
        "散弾・拡散矢強化", "装填拡張", "速射強化", "攻撃"
    ]
}

# 全スキルを集めたリスト
SKILL_MASTER_LIST = [skill for skills in SKILL_DATA.values() for skill in skills]

#　pip

In [ ]:
!pip install easyocr

# # importと定数定義

In [ ]:


import cv2
import difflib
import easyocr
import glob
import json
import numpy as np
import os
import time
import zipfile
from pathlib import Path
import gradio as gr
import json
import shutil
import torch

FILTERD_DIR = "results"
FILTERD_ZIP_DIR = FILTERD_DIR + ".zip"
SETTINGS_JSON_PATH = "settings.json"

# google driveにマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ocr処理を行う関数

In [ ]:
def correct_skill_name(ocr_text, master_list):
    """
    OCRテキストをマスターリスト内の最も近いスキル名に補正します。
    """
    # get_close_matches: 最も似ている候補をリストで返す
    # 0.45は小さすぎるかも
    matches = difflib.get_close_matches(ocr_text, master_list, n=1, cutoff=0.45)
    if matches:
        # 最も似ている候補を返す
        return matches[0]
    else:
        # 適切な候補が見つからない場合は該当なしと返す
        return "該当なし"

def detect_skills_easyocr(image_paths, allowlist):
    # 1. リーダーは関数の外、またはループの直前で1回だけ初期化
    reader = easyocr.Reader(['ja', 'en'], gpu=True)
    image_skills_list = []

    if not os.path.exists(FILTERD_DIR):
        os.makedirs(FILTERD_DIR)

    total_images = len(image_paths)
    CHUNK_SIZE = 12

    # 2. まとめて読み込まず、必要な分だけその都度読み込む
    for i in range(0, total_images, CHUNK_SIZE):
        batch_paths = image_paths[i : i + CHUNK_SIZE]
        current_chunk_images = []

        for p in batch_paths:
            img = cv2.imread(p)
            if img is not None:
                roi = img[361: 565, 540: 700]
                roi = cv2.resize(roi, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                current_chunk_images.append(roi)

        # OCR実行

        results_chunk = reader.readtext_batched(
                current_chunk_images, allowlist=allowlist, batch_size=4, detail=0, paragraph=False
            )

        for j, results in enumerate(results_chunk):
            file_idx = i + j
            skills = [correct_skill_name(text.strip(), SKILL_MASTER_LIST)
            for text in results if len(text) > 1]
            image_skills_list.append([image_paths[file_idx], skills])

        # 3. 非常に重要：VRAMの定期解放
        torch.cuda.empty_cache()

        if i % 120 == 0: # 120枚ごとに進捗表示
             gr.Info(f"OCR実行中... ({i}/{total_images})")

    return image_skills_list


def filter_images_by_skills(image_zip_path):
    gr.Info("画像zipファイルを解凍中")
    extract_dir = "unzipped_img"
    with zipfile.ZipFile(image_zip_path, 'r') as zf:
        zf.extractall(path=extract_dir)
    image_paths = glob.glob(os.path.join(extract_dir, "**/*.jpg"), recursive=True)
    image_paths.sort()

    # jsonファイルを読み込み，欲しいスキルを集めたリストを作成
    with open(SETTINGS_JSON_PATH, "r", encoding="utf-8") as f:
        desired_skill_settings = json.load(f)
        setting_name = "設定1"
        desired_skills = desired_skill_settings[setting_name]
    # 重複を除去して、使われている文字だけの文字列を作る
    #unique_chars = "".join(desired_skills)
    unique_chars = "".join((list(set("".join(SKILL_MASTER_LIST)))))
    gr.Info("画像認識開始")
    for file_no, image_skills_pairs in enumerate(detect_skills_easyocr(image_paths, unique_chars), 1):
        image_path = image_skills_pairs[0]
        skills = image_skills_pairs[1]
        for skill in skills:
            if skill in desired_skills:
                cv2.imwrite(f'results/{str(file_no)}.jpg', cv2.imread(image_path))
                break

# Gradio部分

In [ ]:
# スキル表示を見やすく変更
# Gradio部分

# --- 1ステップ目の処理 ---
def step_1_process(*selected_skills):
    # すべてのグループの選択結果がタプルで届くので、1つのリストに統合
    flatten_list = [skill for group in selected_skills for skill in group]

    if not flatten_list:
        # 選択がない場合は次を表示しない（戻り値：visible設定, 保存用データ）
        return gr.update(visible=False)

    # JSONに保存
    skill_settings = {"設定1": flatten_list}
    with open(SETTINGS_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(skill_settings, f, indent=4, ensure_ascii=False)

    # 次の工程を表示し、リストをStateに渡す
    return gr.update(visible=True)

# --- 2ステップ目の処理 (OCR) ---
def step_2_process(zip_file_name, progress=gr.Progress()):
    if zip_file_name is None:
        raise gr.Error("ファイル名を入力して下さい")

    # Googleドライブ全体からzipファイルを検索
    target_file = zip_file_name + ".zip"
    found = False
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        if target_file in files:
            full_path = os.path.join(root, target_file)
            zip_file = full_path
            found = True
            break

    if not found:
        raise gr.Error(f"エラー: 指定されたファイル '{target_file}' が見つかりません！")

    # ocr 実行
    filter_images_by_skills(zip_file)
    # 3. 圧縮フェーズ
    gr.Info("ファイルを生成しています")
    progress(0.7, desc="ZIPファイルをアーカイブ中...")
    output_zip_path = shutil.make_archive(
        Path(FILTERD_ZIP_DIR).stem,
        format='zip',
        root_dir=FILTERD_DIR
    )

    # 4. 完了
    gr.Info("処理終了")
    return gr.update(visible=True), gr.update(visible=True)

def handle_start_choice(choice):
    # 「持っている」ならアップロード欄を表示、工程1は隠す
    if choice == "はい":
        return gr.update(visible=True), gr.update(visible=False)
    # 「持っていない」なら工程1を表示、アップロード欄は隠す
    return gr.update(visible=False), gr.update(visible=True)

def load_json_config(file_obj):
    if file_obj is None:
        # 空のリストを各グループ分返す + step_1を表示
        return [gr.update(value=[]) for _ in SKILL_DATA] + [gr.update(visible=True)]

    try:
        with open(file_obj.name, "r", encoding="utf-8") as f:
            config = json.load(f)
        selected_items = config.get("設定1", [])

        # 各コストグループに値を振り分ける
        updates = []
        for cost, skills in SKILL_DATA.items():
            # このコストグループに含まれるスキルだけを抽出
            group_val = [s for s in selected_items if s in skills]
            updates.append(gr.update(value=group_val))

        return updates + [gr.update(visible=True)]
    except Exception as e:
        raise gr.Error(f"読み込み失敗: {str(e)}")

with gr.Blocks() as demo:
    saved_list_state = gr.State([])
    gr.Markdown("### スキル判定ワークフロー")

    # --- 導入 ---
    has_json = gr.Radio(["いいえ", "はい"], label="設定JSONファイルを持っていますか？")

    # --- JSONアップロードエリア（「はい」の時だけ表示） ---
    with gr.Column(visible=False) as json_upload_area:
        json_file = gr.File(label="JSONファイルをアップロード", file_types=[".json"])
        btn_load = gr.Button("設定を反映する")

    # --- 工程1（共通の編集エリア） ---
    # JSON読み込み後、ここが自動でチェックされた状態になります
    css = """
    .checkbox-grid .gradio-checkbox-group {
    display: grid;
    grid-template-columns: repeat(3, 1fr); /* 3列に分割 */
    gap: 10px;
    }
    """
    checkbox_groups = []
    with gr.Column(visible=False) as step_1_area:
        gr.Markdown("#### 1. 判定対象スキルの選択")
        for cost, skills in SKILL_DATA.items():
            with gr.Group():
                gr.Markdown(f"**{cost}**")
                cb = gr.CheckboxGroup(choices=skills, label=None, container=False)
                checkbox_groups.append(cb)

        btn1 = gr.Button("この設定で決定", variant="primary")

    # --- 工程2 ---
    with gr.Column(visible=False) as next_step:
        gr.Markdown("---")
        text_input = gr.Textbox(
        label="google driveに画像zipファイルをアップロードし，ファイル名を入力してください（.zipは要りません）",
        lines=3,
        placeholder="ここにファイル名を入力してください..."
    )
        btn2 = gr.Button("画像認識を実行")

    # 工程3
    btn3 = gr.DownloadButton("画像フォルダ（zip）をダウンロード", value=FILTERD_ZIP_DIR, visible=False)
    btn4 = gr.DownloadButton("jsonファイルをダウンロード（次回からスキルの読み込みに使えます）", value=SETTINGS_JSON_PATH, visible=False)

    # --- イベント制御 ---
    # ラジオボタンの選択で、アップロード欄か工程1を出す
    has_json.change(
        fn=lambda x: [gr.update(visible=x=="はい"), gr.update(visible=x=="いいえ")],
        inputs=has_json,
        outputs=[json_upload_area, step_1_area]
    )

    # JSONを読み込んだら、その値をCheckboxGroupにセットして工程1を表示
    btn_load.click(
        fn=load_json_config,
        inputs=json_file,
        outputs=checkbox_groups + [step_1_area]
    )

    # あとは共通：決定ボタンで工程2へ
    btn1.click(
        fn=step_1_process,
        inputs=checkbox_groups,
        outputs=next_step
    )
    btn2.click(
        fn=step_2_process,
        inputs=text_input,
        outputs=[btn3, btn4]
    )

demo.launch(share=True, allowed_paths=["./"])
